In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd


class StockTradingEnv(gym.Env):
    """
    Кастомная среда для симуляции торгов на фондовом рынке.
    Реализует логику MDP (Марковского процесса принятия решений).
    """
    metadata = {"render_modes": ["human"]}

    def __init__(self, df, config):
        super(StockTradingEnv, self).__init__()

        # --- Параметры Конфигурации ---
        self.df = df
        self.cfg = config
        self.n_stocks = self.cfg["n_stocks"]
        self.n_days_history = self.cfg["n_days_history"]
        self.action_space = spaces.Box(
            low=-1, high=1, shape=(self.cfg["n_stocks"],), dtype=np.float32
        )

        # Кол-во признаков по каждой акции: Все признаки самой акции; - 1 id Акции; + 3 Параметра баланса в Портфеле
        self.n_features = len(df.columns) - 1
        self.observation_space = spaces.Box(
            low = - np.inf,
            high = np.inf,
            shape = (
                self.cfg['n_stocks'],
                self.n_features,
                self.cfg
            )
        )


    def __init__(self, df, config):
        super(StockTradingEnv, self).__init__()
        
        # --- Параметры Конфигурации ---
        self.df = df # Весь датасет с признаками
        self.cfg = config['model_light']
        self.prog_cfg = config['progress']
        
        self.n_stocks = self.cfg['n_stocks']
        self.n_days_history = self.cfg['n_days_history']
        self.start_capital = self.cfg['start_capital']
        
        # Комиссии (экранированные значения согласно Принципу №7)
        self.comm_buy = self.cfg['commission_on_buy']
        self.comm_sell = self.cfg['commission_on_sale']
        self.comm_day = self.cfg['commission_on_new_day']

        # --- Определение Пространств (Spaces) ---
        
        # Action Space: Доля капитала для каждого стока (-1 до 1)
        # Мы используем Multi-head Attention, поэтому агент будет выдавать веса для N бумаг
        self.action_space = spaces.Box(
            low=-1, high=1, shape=(self.n_stocks,), dtype=np.float32
        )

        # Observation Space: [Кол-во акций, Окно истории, Кол-во признаков]
        # Примечание: Мы добавляем +1 к признакам для передачи мета-данных портфеля
        n_features = len([
            'open', 'high', 'low', 'close', 'volume', 'adj_close', 
            'close_mean_3d', 'close_std_3d', 'close_mean_30d', 'close_std_30d',
            'volume_mean_7d', 'log_return_1d', 'price_range_1d'
        ])
        
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, 
            shape=(self.n_stocks, self.n_days_history, n_features), 
            dtype=np.float32
        )

        # --- Внутренние переменные состояния ---
        self.current_step = 0
        self.balance = self.start_capital
        self.shares_held = np.zeros(self.n_stocks)
        self.history_variance = [] # Для логики выхода по дисперсии

    def reset(self, seed=None, options=None):
        """
        Сброс среды к начальному состоянию. 
        Вызывается в начале каждого эпизода обучения.
        """
        super().reset(seed=seed)
        
        # Инциализация баланса и шагов
        self.balance = self.start_capital
        self.current_step = self.n_days_history
        self.shares_held = np.zeros(self.n_stocks)
        self.history_variance = []
        
        # Выбор случайных акций для эпизода (Lighter Model constraint)
        self.active_symbols = np.random.choice(
            self.df['symbol'].unique(), self.n_stocks, replace=False
        )
        
        observation = self._get_obs()
        info = {} # Доп. информация для отладки
        
        return observation, info

    def step(self, action):
        """
        Переход среды из состояния S в S+1 под воздействием действия A.
        """
        # 1. Текущие цены закрытия (для оценки портфеля)
        current_prices = self._get_current_prices()
        
        # 2. Логика исполнения ордеров (Execution)
        # Действие агента интерпретируем как желаемую аллокацию портфеля
        self._trade(action, current_prices)
        
        # 3. Переход на следующий временной шаг
        self.current_step += 1
        
        # 4. Применение комиссии за перенос позиции (Overnight/New day)
        self.balance -= self.balance * self.comm_day
        
        # 5. Расчет вознаграждения (Reward)
        # В SAC важно, чтобы Reward был масштабирован. Используем логарифмическую доходность.
        new_net_worth = self._get_net_worth(current_prices)
        reward = np.log(new_net_worth / (self.last_net_worth + 1e-8))
        self.last_net_worth = new_net_worth
        
        # 6. Проверка условий завершения (Termination)
        terminated = self.current_step >= (len(self.df) // self.n_stocks) - 1
        
        # Кастомная логика выхода по дисперсии (variance из конфига)
        self.history_variance.append(new_net_worth)
        truncated = False
        if len(self.history_variance) >= self.cfg['n_tries']:
            recent_var = np.var(self.history_variance[-self.cfg['n_tries']:])
            if recent_var < self.cfg['variance']:
                truncated = True # Ранняя остановка, если эквити "умерло"

        observation = self._get_obs()
        
        return observation, reward, terminated, truncated, {"net_worth": new_net_worth}

    def _get_obs(self):
        """
        Формирует тензор наблюдений для Transformer модели.
        Извлекает срез данных по каждой активной акции за n_days_history.
        """
        obs_matrix = []
        for symbol in self.active_symbols:
            # Извлекаем окно данных для конкретного тикера
            stock_data = self.df[self.df['symbol'] == symbol].iloc[
                self.current_step - self.n_days_history : self.current_step
            ]
            # Удаляем нечисловые колонки (date, symbol)
            features = stock_data.drop(columns=['symbol', 'date']).values
            obs_matrix.append(features)
            
        return np.array(obs_matrix, dtype=np.float32)

    def _trade(self, action, prices):
        """
        Механика покупки/продажи. Обновляет self.balance и self.shares_held.
        """
        for i, weight in enumerate(action):
            # weight > 0: Хотим купить/держать
            # weight < 0: Хотим продать
            if weight > 0:
                # Сколько максимально можем купить на выделенную долю
                amount_to_spend = self.balance * weight
                shares_to_buy = amount_to_spend / (prices[i] * (1 + self.comm_buy))
                self.shares_held[i] += shares_to_buy
                self.balance -= amount_to_spend
            elif weight < 0:
                # Продаем процент от имеющихся акций
                shares_to_sell = self.shares_held[i] * abs(weight)
                gain = shares_to_sell * prices[i] * (1 - self.comm_sell)
                self.shares_held[i] -= shares_to_sell
                self.balance += gain

    def _get_net_worth(self, prices):
        return self.balance + np.sum(self.shares_held * prices)

    def _get_current_prices(self):
        # Получение цен закрытия всех активных акций на текущий шаг
        prices = []
        for symbol in self.active_symbols:
            price = self.df[self.df['symbol'] == symbol].iloc[self.current_step]['close']
            prices.append(price)
        return np.array(prices)

### Создание инвестиционного портфеля

Что в нем должно быть:
- Возможность последовательного проведения транзакций
- Строгое ограничение в кол-ве акций над которыми проводятся операции
- Сохранение истории статистики по открываемым кошелькам


### Проблема покупки долей одновременно
Так как покупка/продажа акций происходит на одновременно компьютером, а последовательно, то при получении от RL модели следующей записи:
```
"Apple": +30% от текущего портфеля
"Google": +50% от текущего портфеля
"Oracle": +60% от текущего портфеля
"Microsoft": -50% от существующего портфеля
```
В этом случае все акции будут покупаться последовательно, и следовательно проценты будут меняться

Более оптимальный вариант:
- Сналача происходит операция продажи указываемых акций
- Полученные деньги прибавляются к текущему балансу
- После этого моделью указывается Процент от текущего портфеля, на который будут закупаться дальнейшие акции

Методика коррекций:

- Получаем от RL модели число, к примеру, 0.5 => 50% от текущего Баланса будет покупаться
- Подается входной массив процентов `a = (0.9, 0.4, 0.2, 0.35)`
- Считается сумма всех элементов. `sum(a) = 1.85`
- Считаем реальную долю каждого элемента, деля каждое число на сумму `a = (0.486, 0.216, 0.108, 0.189)`


In [6]:
a = (0.9, 0.4, 0.2, 0.35)
print(sum(a), [round(x / sum(a), 3) for x in a])

1.85 [0.486, 0.216, 0.108, 0.189]


In [ ]:
from typing import Dict, List, Tuple

import numpy as np
from typing import Dict, List, Tuple

class TradingWallet:
    """
    Оптимизированный портфель для RL-агента.
    Использует принцип разделения фаз (Sell-First) и нормализацию весов.
    """
    def __init__(self):
        self.history_wallets = []
        self.stocks = {}
        self.balance = 0.0
        self.start_capital = 0.0
        self.commission_on_buy = 0.0
        self.commission_on_sale = 0.0

    def open_wallet(
        self,
        start_capital: float,
        stock_names: List[str],
        commission_on_buy: float = 0.0003, # 0.03%
        commission_on_sale: float = 0.0003
    ):
        self.start_capital = start_capital
        self.balance = start_capital
        self.commission_on_buy = commission_on_buy
        self.commission_on_sale = commission_on_sale
        
        # Инкапсуляция данных об акциях
        for name in stock_names:
            self.stocks[name] = {
                'cur_value': 0.0,
                'stocks_have': 0.0,
                'money_inv': 0.0
            }

    def stocks_update(self, current_prices: Dict[str, float]):
        """Обновление рыночной стоимости портфеля"""
        for name, price in current_prices.items():
            if name in self.stocks:
                self.stocks[name]['cur_value'] = price
                # Пересчет стоимости позиции: кол-во акций * цена
                self.stocks[name]['money_inv'] = self.stocks[name]['stocks_have'] * price

    def trade_stocks(self, actions: Dict[str, float]):
        """
        Исполнение торговых сигналов от RL-модели.
        actions: {'TICKER': float от -1 до 1}
        """
        # 1. ФАЗА ПРОДАЖИ (Освобождаем ликвидность)
        for name, weight in actions.items():
            if weight < 0:
                amount_to_sell_percent = abs(weight) # Напр. 0.5 (50%)
                stocks_to_sell = self.stocks[name]['stocks_have'] * amount_to_sell_percent
                
                if stocks_to_sell > 0:
                    revenue = stocks_to_sell * self.stocks[name]['cur_value']
                    fee = revenue * self.commission_on_sale
                    
                    self.balance += (revenue - fee)
                    self.stocks[name]['stocks_have'] -= stocks_to_sell
                    self.stocks[name]['money_inv'] = self.stocks[name]['stocks_have'] * self.stocks[name]['cur_value']

        # 2. ФАЗА ПОКУПКИ (Распределение кеша)
        buy_actions = {k: v for k, v in actions.items() if v > 0}
        if buy_actions:
            total_weight = sum(buy_actions.values())
            available_cash = self.balance
            
            for name, weight in buy_actions.items():
                # Нормализация веса (Ваша методика)
                normalized_weight = weight / total_weight
                money_to_spend = available_cash * normalized_weight
                
                if money_to_spend > 0:
                    # Учет комиссии: покупка на (money - commission)
                    effective_money = money_to_spend / (1 + self.commission_on_buy)
                    fee = effective_money * self.commission_on_buy
                    
                    stocks_bought = effective_money / self.stocks[name]['cur_value']
                    
                    self.stocks[name]['stocks_have'] += stocks_bought
                    self.balance -= (effective_money + fee)
                    self.stocks[name]['money_inv'] = self.stocks[name]['stocks_have'] * self.stocks[name]['cur_value']

    def get_total_net_worth(self) -> float:
        """Общая стоимость активов (Кеш + Акции)"""
        stocks_value = sum(s['money_inv'] for s in self.stocks.values())
        return self.balance + stocks_value

from path import Path



In [35]:
import sys
import os
from pathlib import Path

# Это нужно сделать ОДИН РАЗ в начале ноутбука
# Находим корень проекта (на два уровня выше текущей директории ноутбука)
# и добавляем папку src в пути для импорта
project_root = Path.cwd().parent 
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from local_paths import CONFIGS_DIR, FINAL_DATA_DIR
import yaml
config_name = 'models.yaml'

with open(CONFIGS_DIR / config_name, 'r') as file:
    config_params = yaml.safe_load(file)
    config_params = config_params['model_light']  # Выбираем нужный раздел конфигурации

import pandas as pd
df = pd.read_parquet(FINAL_DATA_DIR / 'combined.parquet')

def sample_stocks(df: pd.DataFrame, sample_size: int = 20) -> List[str]:
    """
    Случайная выборка 
    sample_size уникальных акций из датафрейма на случайную дату.
    Если уникальных акций меньше sample_size, возвращает все доступные.
    """
    import random
    rand_date = df['date'].sample(1).values[0]
    df_sample = df[df['date'] == rand_date]
    n_stocks = list(df_sample['symbol'].unique())
    if len(n_stocks) <= sample_size:
        return n_stocks
    return random.sample(n_stocks, sample_size)

random_stocks = sample_stocks(df, sample_size=10)


In [ ]:
tw = TradingWallet()

tw.open_wallet(
    start_capital=100000,
    stock_names=random_stocks,
    commission_on_buy=0.0003,
    commission_on_sale=0.0003
)

tw.stocks

{'MTVA': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0},
 'HVT-A': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0},
 'CYH': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0},
 'JJSF': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0},
 'TCBX': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0},
 'RIOT': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0},
 'GILD': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0},
 'MRCY': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0},
 'TDG': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0},
 'NGL': {'cur_value': 0.0, 'stocks_have': 0.0, 'money_inv': 0.0}}